###Etapa 1: Recopilación y Preparación de Datos (Clases 1 a 4)


0 - Librerías y Funciones Útiles

Objetivo

Configurar el entorno de ejecución cargando las librerías necesarias para el manejo, análisis y visualización de datos, y definir una función personalizada para formatear valores monetarios de manera consistente.

Proceso
1. Importación de Librerías: Se cargan pandas (para DataFrames), os (para manejo de rutas) e IPython.display (para visualización en Colab).
2. Definición de Función formato_moneda: Se crea una función que toma un valor numérico, lo formatea a dos decimales y ajusta los separadores (punto para miles, coma para decimales) para mostrarlo en formato de moneda en español ($ 12.345,67).

Resultado

Un entorno listo para el procesamiento de datos y una función de formato reutilizable disponible para todos los DataFrames y análisis.

In [1]:
# =====================================================
# MÓDULO 0 : Librerías y Funciones Útiles
# =====================================================

# LIBRERÍAS: Son herramientas externas que añaden funcionalidades.
from google.colab import drive # Permite conectar Colab (que está en la nube) con tu Google Drive.
import os                      # Módulo para interactuar con el sistema operativo, útil para manejar rutas de archivos.
import pandas as pd            # La librería más importante: se usa para trabajar con datos tabulares (DataFrames).
from IPython.display import display # Función para mostrar DataFrames de Pandas de forma limpia en Colab.

# FUNCIÓN DE FORMATO DE MONEDA: Una función personalizada.
# Definimos esta función una vez para reutilizarla en todos los precios/ingresos.
def formato_moneda(valor):
    """
    Esta función toma un número (valor) y lo convierte en un texto con formato de moneda en español.
    Ejemplo: 12345.67 se convierte en "$ 12.345,67".
    """
    # 1. Formateamos el número a dos decimales y usamos la coma (,) como separador de miles.
    # 2. Luego, cambiamos los separadores al estándar español (punto para miles, coma para decimales).
    # 3. Se añade el símbolo '$ ' al inicio.
    formato_limpio = "{:,.2f}".format(valor).replace(",", "X").replace(".", ",").replace("X", ".")
    return f"$ {formato_limpio}"

# SALIDA POR PANTALLA: Confirmación de que el primer módulo se ejecutó con éxito.
print("Módulo 0 completado: Librerías importadas y función 'formato_moneda' definida y lista para usar.")

Módulo 0 completado: Librerías importadas y función 'formato_moneda' definida y lista para usar.


1 - Crear un documento en Google Colaboratory y cargar los sets de datos como DataFrames.



Objetivo

Establecer la conexión, cargar los sets de datos y aplicar una limpieza y conversión de tipos básica pero completa, ya que estos pasos son un requisito indispensable para poder realizar los cálculos matemáticos y operaciones temporales exigidos en el Enunciado 2 de la etapa 1(Ventas Mensuales).

Proceso
1. Módulos 1.1 y 1.2 (Carga): Se monta Google Drive (drive.mount()), se verifica la ruta y se cargan los tres archivos (marketing, clientes, ventas) en DataFrames de Pandas (pd.read_csv()).
2. Módulo 1.3 (Respaldo): Se crean copias inalteradas (.copy()) de los DataFrames originales (df_marketing_diag, etc.) para usarlos como respaldo y para el diagnóstico futuro.
3. Módulo 1.4 (Limpieza y Tipos Básicos): (Paso de Habilitación para Enunciado 2) Se eliminan duplicados y nulos. Se estandarizan ruidos (VALORES_MALOS -> pd.NA) y, lo más importante, se convierten las columnas clave a su tipo correcto (float para costos/precios/ingresos, datetime para fechas, int para edad/cantidad), recalculando el ingreso_bruto.
4. Módulo 1.5 (Visualización): Se aplica el formato de moneda y fecha (DD-MM-YYYY) a las visualizaciones de los DataFrames limpios (.style.format()) para confirmar el éxito de la conversión de tipos.

Resultado

DataFrames cargados, verificados, respaldados y listos para realizar cálculos matemáticos y temporales. Se confirma que las columnas financieras y de fecha tienen el tipo de dato adecuado para el siguiente enunciado.

In [2]:
# =====================================================
# MÓDULO 1.1 : Montar Google Drive y Verificar Rutas
# =====================================================

# Montar Drive. Esto iniciará el proceso de autenticación para que Google Colab acceda a tu Drive.
# La función drive.mount() es crucial para establecer la conexión con la ruta local /content/drive.
drive.mount("/content/drive", force_remount=True)

# Definimos la ruta exacta de la carpeta donde se encuentran los archivos CSV.
ruta_datasets = "/content/drive/MyDrive/Datasets"

# Verificación de la ruta: Es el primer paso de control de calidad.
if os.path.exists(ruta_datasets):
    print("Carpeta Datasets encontrada correctamente!")

    # Lista de archivos: Se utiliza os.listdir() para obtener una lista de los nombres de todos los archivos.
    archivos = os.listdir(ruta_datasets)

    # SALIDA POR PANTALLA: Ahora se imprimen uno debajo del otro usando un bucle.
    print("\n Archivos encontrados:")
    # Usamos un bucle for: por cada 'archivo' que esté en la lista 'archivos', imprímelo.
    # Cada print() agrega automáticamente un salto de línea.
    for archivo in archivos:
        print(f"  - {archivo}") # f"..." permite incrustar la variable archivo y usar un guion/espacio.

    print("\n-------------------------------------------")
else:
    # Si la ruta es incorrecta, se detiene la ejecución con un error.
    raise FileNotFoundError("ERROR: La carpeta Datasets no se encontró. Verifica que la ruta y el nombre sean exactos.")

Mounted at /content/drive
Carpeta Datasets encontrada correctamente!

 Archivos encontrados:
  - marketing.csv
  - clientes.csv
  - ventas.csv

-------------------------------------------


In [3]:
# =====================================================
# MÓDULO 1.2 : Cargar CSVs en DataFrames
# =====================================================

# El objetivo de esta celda es leer los archivos .csv desde la ruta que verificamos en el Módulo 1,
# y convertirlos en objetos de Pandas llamados DataFrames (tablas en memoria).

# 1. Construimos la ruta completa (path) para cada archivo CSV.
# Esto asegura que la ubicación sea precisa para la función de lectura de Pandas.
# os.path.join() maneja automáticamente el tipo de barra separadora (/, \) de tu sistema operativo.
archivo_marketing = os.path.join(ruta_datasets, "marketing.csv")
archivo_clientes = os.path.join(ruta_datasets, "clientes.csv")
archivo_ventas = os.path.join(ruta_datasets, "ventas.csv")

# 2. Leemos cada archivo CSV usando pd.read_csv() y los asignamos a variables de DataFrames (df_).
# pd.read_csv es la función de Pandas para convertir el texto separado por comas en una estructura de tabla.
df_marketing = pd.read_csv(archivo_marketing)
df_clientes = pd.read_csv(archivo_clientes)
df_ventas = pd.read_csv(archivo_ventas)

# SALIDA POR PANTALLA: Se muestra un resumen de los tipos de datos iniciales.
print("Módulo 2 completado: DataFrames cargados en memoria.")

# Usamos .info() para una inspección rápida. Es una herramienta clave de Pandas para:
# 1. Confirmar el número total de filas (Entries).
# 2. Ver cuántos valores no-nulos (non-null) tiene cada columna (ideal para detectar datos faltantes).
# 3. Mostrar el tipo de dato que Pandas asignó automáticamente (Dtype).
print("\nTipos de datos iniciales de df_marketing (info() muestra el conteo de datos y el Dtype):")
# verbose=False muestra solo un resumen, sin la lista detallada de columnas.
df_marketing.info(verbose=False, buf=None)

Módulo 2 completado: DataFrames cargados en memoria.

Tipos de datos iniciales de df_marketing (info() muestra el conteo de datos y el Dtype):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 6 entries, id_campanha to fecha_fin
dtypes: float64(1), int64(1), object(4)
memory usage: 4.3+ KB


In [4]:
# =====================================================
# MÓDULO 1.3: Preparación de DataFrames para Limpieza
# =====================================================

print("\nIniciando Módulo 1.3: Creando copias de DataFrames para Diagnóstico y Respaldo...")

# --------------------------------------------------------------------------------------
# OBJETIVO: Crear copias de los DataFrames originales antes de cualquier transformación.
# Esto nos permite:
# 1. Ejecutar el Módulo 5 (Diagnóstico) sobre las copias para ver el estado inicial.
# 2. Mantener los DataFrames originales inalterados por si necesitamos revertir la limpieza.
# --------------------------------------------------------------------------------------

# Utilizamos el método .copy() para hacer una copia profunda (deep copy).
# Esto significa que los DataFrames originales y las copias son completamente independientes
# y los cambios hechos en uno NO afectarán al otro.

# Copia para Diagnóstico de df_marketing
df_marketing_diag = df_marketing.copy()

# Copia para Diagnóstico de df_clientes
df_clientes_diag = df_clientes.copy()

# Copia para Diagnóstico de df_ventas
df_ventas_diag = df_ventas.copy()

print("Módulo 1.3 completado: DataFrames de diagnóstico ('_diag') creados correctamente.")


Iniciando Módulo 1.3: Creando copias de DataFrames para Diagnóstico y Respaldo...
Módulo 1.3 completado: DataFrames de diagnóstico ('_diag') creados correctamente.


In [5]:
# =====================================================
# MÓDULO 1.4: Limpiar y mejorar DataFrames (Transformación de datos)
# =====================================================

# El objetivo de este módulo es asegurar que todas las columnas clave tengan el tipo de dato correcto
# para poder realizar cálculos matemáticos (con floats/ints) y operaciones temporales (con datetime).

# Definimos los valores que se consideran "malos" o "vacíos" y que deben ser convertidos a NaN.
VALORES_MALOS = ['', ' ', '-', 'N/A', 'N/D', 'na']


# ---- DataFrame df_marketing ----
print("Limpiando df_marketing...")

# =========================================================================
# Deduplicación, Ruidos y Nulos
# =========================================================================
# 1. Deduplicación: Eliminamos las filas que son copias exactas en todas sus columnas.
#    Esto es crucial para la integridad de los datos y evitar sesgos en métricas como el 'Conteo'.
filas_iniciales_m = len(df_marketing)
df_marketing.drop_duplicates(inplace=True)
print(f"  -> Se eliminaron {filas_iniciales_m - len(df_marketing)} filas duplicadas.")

# 2. Reemplazo de Ruidos/Vacíos: Convertimos valores como 'N/A', '', etc., a NaN.
#    Esta es la etapa para estandarizar todos los datos defectuosos (ruidosos, vacíos)
#    a un único formato (pd.NA) para que el 'dropna' los pueda detectar.
df_marketing.replace(VALORES_MALOS, pd.NA, inplace=True)

# 3. Eliminación de Nulos (Registros Incompletos): Borramos cualquier fila con algún valor nulo (NaN).
#    Aseguramos que cada registro de marketing esté completo.
df_marketing.dropna(inplace=True)
# =========================================================================

# Columna 'costo': Convertir a float.
# 'float' es el tipo de dato para números que pueden tener decimales, necesario para cálculos de presupuesto.
df_marketing['costo'] = df_marketing['costo'].astype(float)

# Columnas de fecha: Convertir a datetime.
# 'datetime' es el tipo especial de Pandas que permite calcular duraciones, comparar fechas, etc.
# pd.to_datetime() realiza la conversión, y format='%d/%m/%Y' le indica a Python que la fecha de entrada
# en el archivo original estaba en formato día/mes/año.
df_marketing['fecha_inicio'] = pd.to_datetime(df_marketing['fecha_inicio'], format='%d/%m/%Y')
df_marketing['fecha_fin'] = pd.to_datetime(df_marketing['fecha_fin'], format='%d/%m/%Y')


# ---- DataFrame df_clientes ----
print("Limpiando df_clientes...")

# =========================================================================
# Deduplicación, Ruidos y Nulos
# =========================================================================
# 1. Deduplicación
filas_iniciales_c = len(df_clientes)
df_clientes.drop_duplicates(inplace=True)
print(f"  -> Se eliminaron {filas_iniciales_c - len(df_clientes)} filas duplicadas.")

# 2. Reemplazo de Ruidos/Vacíos: Convertimos valores como 'N/A', '', etc., a NaN.
#    Estandarizamos los valores ruidosos o vacíos para ser tratados como nulos.
df_clientes.replace(VALORES_MALOS, pd.NA, inplace=True)

# 3. Eliminación de Nulos: Borramos cualquier fila que contenga valores nulos (NaN).
df_clientes.dropna(inplace=True)
# =========================================================================

# Columna 'edad': Asegurar que es un número entero (int).
# La edad generalmente no tiene decimales.
df_clientes['edad'] = df_clientes['edad'].astype(int)

# Columna 'ingresos': Asegurar que es un número con decimales (float) para cálculos financieros.
df_clientes['ingresos'] = df_clientes['ingresos'].astype(float)


# ---- DataFrame df_ventas ----
print("Limpiando df_ventas...")

# =========================================================================
# Deduplicación y Manejo Específico de Nulos
# =========================================================================
# 1. Deduplicación
filas_iniciales_v = len(df_ventas)
df_ventas.drop_duplicates(inplace=True)
print(f"  -> Se eliminaron {filas_iniciales_v - len(df_ventas)} filas duplicadas.")

# 2. Manejo Específico de Nulos:
#    - Reemplazo de Ruidos/Vacíos: Convertimos ruidos a NaN antes del conteo.
df_ventas.replace(VALORES_MALOS, pd.NA, inplace=True)

#    - Pre-conteo: Guardamos la cantidad de filas antes de eliminar los nulos.
filas_antes_nulos = len(df_ventas)

#    - Eliminar nulos en 'precio': Borramos las filas donde 'precio' es nulo.
#      Esto es esencial porque un precio nulo impide el cálculo de ingresos y la columna es crítica.
df_ventas.dropna(subset=['precio'], inplace=True)

#    - Post-conteo: Calculamos cuántas filas se perdieron debido a los nulos.
filas_eliminadas_por_nulo = filas_antes_nulos - len(df_ventas)
print(f"  -> Se eliminaron {filas_eliminadas_por_nulo} registros por tener 'precio' nulo (NaN) o vacío.")
# =========================================================================

# Columna 'precio': Limpiar y convertir a float.
# 1. .replace(r'[\$,]', '', regex=True): Buscamos el símbolo '$' o la coma de miles (si existe) y lo eliminamos.
#    'regex=True' indica que usamos expresiones regulares para encontrar el patrón.
# 2. .astype(float): Una vez limpia, la convertimos a un número con decimales.
df_ventas['precio'] = df_ventas['precio'].replace(r'[\$,]', '', regex=True).astype(float)

# Columna 'cantidad': Rellenar nulos (si los hay) y convertir a entero (int).
# 1. .fillna(0): Si falta un valor de cantidad, asumimos que es 0 (para evitar errores en la conversión).
# 2. .astype(int): Convertimos el resultado a un número entero.
df_ventas['cantidad'] = df_ventas['cantidad'].fillna(0).astype(int)

# Columna 'fecha_venta': Convertir a datetime.
df_ventas['fecha_venta'] = pd.to_datetime(df_ventas['fecha_venta'], format='%d/%m/%Y')

# CÁLCULO FINAL: Recalculamos el ingreso bruto.
# Es buena práctica asegurar que esta columna se genere DESPUÉS de toda la limpieza de precio y cantidad.
df_ventas['ingreso_bruto'] = df_ventas['precio'] * df_ventas['cantidad']


# SALIDA POR PANTALLA: Mostrar los tipos de datos después de la limpieza para confirmar el éxito.
print("\nMódulo 3 completado: Limpieza, deduplicación y conversión de tipos finalizada.")
print("\nTipos de datos finales de df_ventas (Verificación después de la limpieza):")
# Mostramos la info del DataFrame de ventas para confirmar que 'precio' y 'fecha_venta' son float y datetime.
df_ventas.info(verbose=False, buf=None)

Limpiando df_marketing...
  -> Se eliminaron 0 filas duplicadas.
Limpiando df_clientes...
  -> Se eliminaron 0 filas duplicadas.
Limpiando df_ventas...
  -> Se eliminaron 35 filas duplicadas.
  -> Se eliminaron 2 registros por tener 'precio' nulo (NaN) o vacío.

Módulo 3 completado: Limpieza, deduplicación y conversión de tipos finalizada.

Tipos de datos finales de df_ventas (Verificación después de la limpieza):
<class 'pandas.core.frame.DataFrame'>
Index: 2998 entries, 0 to 3034
Columns: 7 entries, id_venta to ingreso_bruto
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 187.4+ KB


In [6]:
# =====================================================
# MÓDULO 1.5: Mejorar visualización en Colab y Mostrar Resultados
# =====================================================

# Este módulo se encarga únicamente de la presentación final de los DataFrames,
# aplicando los formatos definidos en el Módulo 0 (formato_moneda y formato de fecha DD-MM-YYYY).
# ¡Importante! Esto SÓLO afecta la forma en que se ve la tabla, no los datos subyacentes.

# Configuraciones de Pandas para mejorar la presentación de las tablas.
pd.set_option('display.max_columns', None) # Muestra TODAS las columnas sin cortar.
pd.set_option('display.width', 120) # Define el ancho máximo de la tabla.
pd.set_option('display.colheader_justify', 'left') # Alinea los encabezados a la izquierda.

print("Módulo 4: Aplicando estilos de visualización (dd-mm-yyyy y moneda contable) a los DataFrames.")

# --- Marketing ---
print("\n--- DataFrame Marketing con Formato (fechas y costos) ---")
# Usamos .style.format() para aplicar formatos SIN cambiar los datos originales.
df_marketing_estilo = df_marketing.head(5).style.format({
    'costo': formato_moneda,                       # Aplica el formato de moneda (ej: 4.81 €).
    # lambda x: x.strftime('%d-%m-%Y') convierte el objeto datetime a la cadena deseada (ej: 20-03-2024).
    'fecha_inicio': lambda x: x.strftime('%d-%m-%Y'),
    'fecha_fin': lambda x: x.strftime('%d-%m-%Y')
})
display(df_marketing_estilo) # Mostramos el DataFrame con el nuevo estilo.

# --- Clientes ---
print("\n--- DataFrame Clientes con Formato (ingresos) ---")
df_clientes_estilo = df_clientes.head(5).style.format({
    'ingresos': formato_moneda, # Aplicamos formato de moneda a los ingresos.
})
display(df_clientes_estilo)

# --- Ventas ---
print("\n--- DataFrame Ventas con Formato (precio y fecha) ---")
# La corrección para la fecha de ventas se asegura de que el formato DD-MM-YYYY se aplique aquí.
df_ventas_estilo = df_ventas.head(5).style.format({
    'precio': formato_moneda,                     # Aplicamos formato de moneda al precio.
    'fecha_venta': lambda x: x.strftime('%d-%m-%Y') # Aplica el formato de visualización Día-Mes-Año.
})
display(df_ventas_estilo)

Módulo 4: Aplicando estilos de visualización (dd-mm-yyyy y moneda contable) a los DataFrames.

--- DataFrame Marketing con Formato (fechas y costos) ---


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,"$ 4,81",20-03-2024,03-05-2024
1,12,Tablet,RRSS,"$ 3,40",26-03-2024,13-05-2024
2,32,Lámpara de mesa,Email,"$ 5,54",28-03-2024,20-04-2024
3,21,Smartphone,RRSS,"$ 6,37",29-03-2024,16-05-2024
4,58,Alfombra,Email,"$ 4,25",31-03-2024,05-05-2024



--- DataFrame Clientes con Formato (ingresos) ---


,id_cliente,nombre,edad,ciudad,ingresos
0,1,Aloysia Screase,44,Mar del Plata,"$ 42.294,68"
1,2,Kristina Scaplehorn,25,Posadas,"$ 24.735,04"
2,3,Filip Castagne,50,Resistencia,"$ 35.744,85"
3,4,Liuka Luard,39,Bahía Blanca,"$ 27.647,96"
4,5,Dore Cockshtt,28,Rosario,"$ 28.245,65"



--- DataFrame Ventas con Formato (precio y fecha) ---


,id_venta,producto,precio,cantidad,fecha_venta,categoria,ingreso_bruto
0,792,Cuadro decorativo,"$ 69,94",5,02-01-2024,Decoración,349.700000
1,811,Lámpara de mesa,"$ 105,10",5,02-01-2024,Decoración,525.500000
2,1156,Secadora,"$ 97,96",3,02-01-2024,Electrodomésticos,293.880000
3,1372,Heladera,"$ 114,35",8,02-01-2024,Electrodomésticos,914.800000
4,1546,Secadora,"$ 106,21",4,02-01-2024,Electrodomésticos,424.840000


2 - Realizar un script básico que calcule las ventas mensuales utilizando variables y operadores.



Objetivo

Calcular las dos métricas fundamentales del rendimiento de ventas (Ingreso Bruto Total y Unidades Totales Vendidas) utilizando los datos limpios de df_ventas y operadores de Python/Pandas.

Proceso
1. Cálculo por Línea: Se utiliza el operador de multiplicación (*) para crear la columna intermedia 'ingreso_bruto' multiplicando 'precio' por 'cantidad' en cada fila.
2. Agregación Global: Se utiliza el método de agregación .sum() para obtener el Ingreso Bruto Total del periodo (sumando la nueva columna 'ingreso_bruto') y el Total de Unidades Vendidas (sumando la columna 'cantidad').
3. Presentación: Se aplica la función formato_moneda (definida en el Módulo 0) a la variable de Ingreso Bruto y se utiliza formato de cadenas (f-strings) para generar un reporte final claro y legible.

Resultado

Se obtienen y se presentan dos variables clave que resumen el desempeño financiero del periodo: el Ingreso Bruto Total y la Cantidad Total de Unidades Vendidas, basándose 100% en df_ventas.

In [7]:
# =====================================================
# MÓDULO 2: Cálculo de Variables (Usando df_ventas)
# =====================================================

# El objetivo es calcular las dos métricas fundamentales de ventas (Ingreso Bruto y Unidades)
# utilizando los datos del DataFrame df_ventas.

# PASO 1: Creación de la columna de ingreso por línea.
# Utilizamos el operador de multiplicación (*) para calcular el ingreso individual de cada fila.
# Esto es esencial para cualquier cálculo posterior.
df_ventas['ingreso_bruto'] = df_ventas['precio'] * df_ventas['cantidad']
print("Paso 1: Columna 'ingreso_bruto' (precio * cantidad) calculada en el DataFrame.")

# CÁLCULO DE LA VENTA BRUTA TOTAL (VARIABLE 1)
# Usamos el operador de suma (.sum()) para obtener el total de ingresos del mes.
ventas_brutas_mensuales = df_ventas['ingreso_bruto'].sum()
print("Paso 2: Variable 'ventas_brutas_mensuales' calculada.")

# CÁLCULO DE UNIDADES TOTALES VENDIDAS (VARIABLE 2)
# Sumamos la columna 'cantidad' para obtener el total de productos vendidos.
unidades_totales_vendidas = df_ventas['cantidad'].sum()
print("Paso 3: Variable 'unidades_totales_vendidas' calculada.")

# SALIDA POR PANTALLA: Confirmación de las variables clave obtenidas del DataFrame.
print("\nMódulo 1 completado: Variables de ventas extraídas de df_ventas.")
print(f"   - Ingreso Bruto Total: {ventas_brutas_mensuales:.2f}")
print(f"   - Unidades Totales: {unidades_totales_vendidas}")

Paso 1: Columna 'ingreso_bruto' (precio * cantidad) calculada en el DataFrame.
Paso 2: Variable 'ventas_brutas_mensuales' calculada.
Paso 3: Variable 'unidades_totales_vendidas' calculada.

Módulo 1 completado: Variables de ventas extraídas de df_ventas.
   - Ingreso Bruto Total: 1467093.52
   - Unidades Totales: 19495


In [8]:
# =====================================================
# MÓDULO 2.1: Presentación del Reporte de Ventas
# =====================================================

# El objetivo es mostrar el resultado de las variables calculadas con un formato legible.
# Dependemos de la función 'formato_moneda' (que usa el signo $).

# 1. Aplicamos el formato de moneda a la variable de Ingresos Brutos.
ventas_brutas_formato = formato_moneda(ventas_brutas_mensuales)

# --------------------------------------------------------------------------------
# REPORTE FINAL DE VENTAS BASADO 100% EN DF_VENTAS
# --------------------------------------------------------------------------------
print("\n" + "="*60)
print(f"| {'REPORTE DE VENTAS MENSUALES':^58} |")
print("="*60)

# Mostramos las variables calculadas.
print(f"| {'A) Ingreso Bruto Total del Mes:':<40} {ventas_brutas_formato:>17} |")
print(f"| {'B) Cantidad Total de Unidades Vendidas:':<40} {unidades_totales_vendidas:>17} |")
print("="*60)

# SALIDA POR PANTALLA: Confirmación final.
print("\n Módulo 2 completado: El reporte de ventas basado en df_ventas se ha generado.")


|                REPORTE DE VENTAS MENSUALES                 |
| A) Ingreso Bruto Total del Mes:             $ 1.467.093,52 |
| B) Cantidad Total de Unidades Vendidas:              19495 |

 Módulo 2 completado: El reporte de ventas basado en df_ventas se ha generado.


3 - Estructuras de Datos: Desarrollar un programa que almacene los datos de ventas (producto, precio, cantidad). Decidir si conviene utilizar diccionarios o listas.

Objetivo

Demostrar la extracción de datos clave de una fuente tabular (df_ventas) y su almacenamiento en una estructura de datos nativa de Python (Lista de Diccionarios), justificando por qué esta es la opción más adecuada.
Estructura Elegida	Lista de Diccionarios. Se elige esta estructura porque cada registro de venta (la fila) es una colección de pares clave-valor (el diccionario), y la colección de todos los registros es la Lista. Esto mantiene la semántica de los datos tabulares.

Proceso
1. Inicialización: Se crea una lista vacía, diccionario_ventas.
2. Iteración: Se utiliza el método df_ventas.iterrows() para recorrer el DataFrame fila por fila.
3. Almacenamiento: En cada iteración, se construye un diccionario que extrae explícitamente los valores de 'producto', 'precio' y 'cantidad'.
4. Apéndice: El diccionario recién creado se añade (.append()) a la lista principal.

Resultado

Una Lista de Diccionarios en memoria que almacena de forma eficiente y semánticamente correcta todos los registros de ventas, demostrando cómo estructurar datos tabulares fuera de Pandas.

In [9]:
# =====================================================
# MÓDULO 3: Estructuras de Datos (Lista de Diccionarios)
# =====================================================

# El objetivo es extraer los campos clave (producto, precio, cantidad) del DataFrame df_ventas
# y almacenarlos en una Lista de Diccionarios, la estructura nativa más adecuada para datos tabulares.

# 1. Creamos la estructura de datos: una Lista vacía que contendrá diccionarios.
diccionario_ventas = []
print("Paso 1: Inicializada la Lista vacía 'diccionario_ventas'.")

# Importamos la librería para imprimir estructuras complejas de forma legible.
from pprint import pprint

# 2. Recorremos el DataFrame df_ventas.
# iterrows() nos permite recorrer cada fila (row) del DataFrame.
for index, row in df_ventas.iterrows():
    # 3. Almacenamos los datos de la fila actual en un DICCIONARIO.
    registro_venta = {
        'producto': row['producto'],      # Obtenemos el producto de la fila actual
        'precio': row['precio'],          # Obtenemos el precio de la fila actual
        'cantidad': row['cantidad']       # Obtenemos la cantidad de la fila actual
    }

    # 4. Añadimos el diccionario creado a la LISTA principal.
    diccionario_ventas.append(registro_venta)

# 5. Verificamos el tamaño total de la estructura.
total_registros = len(diccionario_ventas)

# SALIDA POR PANTALLA: Confirmación de que se almacenaron todos los registros.
print(f"Módulo 3 completado: Se almacenaron {total_registros} registros en la lista 'ventas_por_diccionario'.")

# 6. Mostramos solo una muestra de los 10 primeros registros.
print("\n Muestra de los primeros 10 registros en la estructura 'ventas_por_diccionario':")
# Usamos la notación de slicing [0:10] para mostrar solo la muestra.
pprint(diccionario_ventas[0:10], indent=1)

Paso 1: Inicializada la Lista vacía 'diccionario_ventas'.
Módulo 3 completado: Se almacenaron 2998 registros en la lista 'ventas_por_diccionario'.

 Muestra de los primeros 10 registros en la estructura 'ventas_por_diccionario':
[{'cantidad': 5, 'precio': 69.94, 'producto': 'Cuadro decorativo'},
 {'cantidad': 5, 'precio': 105.1, 'producto': 'Lámpara de mesa'},
 {'cantidad': 3, 'precio': 97.96, 'producto': 'Secadora'},
 {'cantidad': 8, 'precio': 114.35, 'producto': 'Heladera'},
 {'cantidad': 4, 'precio': 106.21, 'producto': 'Secadora'},
 {'cantidad': 9, 'precio': 35.35, 'producto': 'Horno eléctrico'},
 {'cantidad': 2, 'precio': 65.43, 'producto': 'Plancha de vapor'},
 {'cantidad': 9, 'precio': 88.17, 'producto': 'Proyector'},
 {'cantidad': 11, 'precio': 79.86, 'producto': 'Rincón de plantas'},
 {'cantidad': 8, 'precio': 66.11, 'producto': 'Candelabro'}]


4 - Introducción a Pandas: realizar un análisis exploratorio inicial de los DataFrames.

Objetivo

Realizar un Análisis Exploratorio de Datos (EDA) inicial, utilizando el método descriptivo de Pandas (.describe()), para obtener una comprensión fundamental de la distribución central, dispersión y rangos de las variables numéricas clave en cada DataFrame.

Proceso
1. Mapeo: Se define un diccionario (mapa_espanol) para traducir los índices estadísticos estándar de Pandas (ej., 'mean', 'std', '50%') a sus equivalentes en español.
2. Análisis Descriptivo (.describe()): Se aplica el método describe() a las columnas numéricas de los tres DataFrames: Marketing (costo), Clientes (edad, ingresos) y Ventas (precio, cantidad, ingreso_bruto).
3. Formato Personalizado: Se crean funciones lambda que se aplican a las tablas descriptivas para asegurar que todos los valores monetarios (costos, ingresos, precios) utilicen la función formato_moneda, mientras que el Conteo se presenta como un entero.
4. Análisis Adicional: Se utiliza value_counts() para examinar las variables categóricas clave (ciudad y categoria), identificando las ubicaciones y los productos con mayor número de registros.

Resultado

Una presentación de tres tablas descriptivas traducidas y formateadas que muestran las ocho estadísticas clave (Media, Desviación Estándar, Mínimo, Máximo y Cuartiles) para las variables numéricas, proporcionando la primera visión estadística de los datos.

In [10]:
# =====================================================
# MÓDULO 4: Análisis Exploratorio Inicial (EDA)
# =====================================================

print("Iniciando Módulo 4: Análisis Exploratorio de DataFrames...")
print("----------------------------------------------------------")

# --- Mapeo para traducir los índices de las tablas descriptivas ---
mapa_espanol = {
    'count': 'Conteo',
    'mean': 'Media',
    'std': 'Desviación Estándar',
    'min': 'Mínimo',
    '25%': 'Cuartil 25%',
    '50%': 'Mediana (50%)',
    '75%': 'Cuartil 75%',
    'max': 'Máximo'
}

# --- 1. Análisis de df_marketing ---
print("\n[A] RESUMEN DE COSTOS DE CAMPAÑA (df_marketing)")
df_marketing_desc = df_marketing[['costo']].describe()

def aplicar_formato_marketing(x):
    if x.name == 'count':
        # Conteo: Se muestra como entero
        return x.apply(lambda val: f"{val:.0f}") # <-- CORRECCIÓN: Aplicar a cada valor de la Serie

    # Resto (Media, Desviación, Cuartiles): Aplicar formato de moneda a cada valor
    return x.apply(formato_moneda) # <-- CORRECCIÓN: Aplicar formato_moneda a cada valor de la Serie

df_marketing_desc = df_marketing_desc.apply(aplicar_formato_marketing)
df_marketing_desc = df_marketing_desc.rename(index=mapa_espanol)
display(df_marketing_desc)

# --- 2. Análisis de df_clientes ---
print("\n[B] RESUMEN DEMOGRÁFICO Y FINANCIERO (df_clientes)")
df_clientes_desc = df_clientes[['edad', 'ingresos']].describe()

def aplicar_formato_clientes(columna_serie):
    if columna_serie.name == 'edad':
        # Edad: Se muestra con dos decimales, excepto el conteo
        return columna_serie.apply(lambda x: f"{x:.0f}" if x.is_integer() else f"{x:.2f}")

    # Ingresos: Usa formato de moneda, excepto el conteo (que es un valor especial)
    # Nota: El conteo de ingresos ya se maneja en 'df_clientes_desc', así que aplicamos el formato de moneda al resto.
    columna_serie_formateada = columna_serie.apply(formato_moneda)

    # Aseguramos que el Conteo de Ingresos sea un número limpio (puede requerir un ajuste específico si el df original era string)
    # Por ahora, simplemente eliminamos el símbolo $ y los decimales del conteo
    columna_serie_formateada['count'] = f"{float(columna_serie['count']):.0f}"

    return columna_serie_formateada

df_clientes_desc = df_clientes_desc.apply(aplicar_formato_clientes)
df_clientes_desc = df_clientes_desc.rename(index=mapa_espanol)
display(df_clientes_desc)


# Análisis adicional de ubicación geográfica
print("\nDISTRIBUCIÓN DE CLIENTES POR CIUDAD (df_clientes['ciudad'].value_counts().head(5)):")
print(df_clientes['ciudad'].value_counts().head(5))


# --- 3. Análisis de df_ventas ---
print("\n[C] RESUMEN DE PRECIOS, CANTIDADES E INGRESOS (df_ventas)")
df_ventas_desc = df_ventas[['precio', 'cantidad', 'ingreso_bruto']].describe()

def aplicar_formato_ventas(x):
    # Conteo: Se muestra como entero
    if x.name == 'count':
        return f"{x:.0f}"
    # Columna 'cantidad': Se muestra con dos decimales, pero sin símbolo de moneda
    elif x.name == 'cantidad':
        return x.apply(lambda y: f"{y:.2f}")
    # 'precio' e 'ingreso_bruto': Se muestra con formato de moneda
    return x.apply(formato_moneda)


df_ventas_desc = df_ventas_desc.apply(aplicar_formato_ventas)
df_ventas_desc = df_ventas_desc.rename(index=mapa_espanol)
display(df_ventas_desc)

# Análisis adicional de productos
print("\nPRODUCTOS MÁS VENDIDOS (POR CANTIDAD DE TRANSACCIONES):")
print(df_ventas['categoria'].value_counts().head(5))

print("\n----------------------------------------------------------")
print("Módulo 4 completado: El formato numérico estricto se ha aplicado.")

Iniciando Módulo 4: Análisis Exploratorio de DataFrames...
----------------------------------------------------------

[A] RESUMEN DE COSTOS DE CAMPAÑA (df_marketing)


,costo
Conteo,"$ 90,00"
Media,"$ 4,93"
Desviación Estándar,"$ 0,95"
Mínimo,"$ 2,95"
Cuartil 25%,"$ 4,37"
Mediana (50%),"$ 4,90"
Cuartil 75%,"$ 5,56"
Máximo,"$ 7,39"



[B] RESUMEN DEMOGRÁFICO Y FINANCIERO (df_clientes)


,edad,ingresos
Conteo,567,567
Media,37.94,"$ 34.668,74"
Desviación Estándar,10.20,"$ 12.974,53"
Mínimo,20,"$ 170,29"
Cuartil 25%,30,"$ 26.015,24"
Mediana (50%),37,"$ 35.066,83"
Cuartil 75%,43,"$ 42.457,10"
Máximo,81,"$ 88.053,01"



DISTRIBUCIÓN DE CLIENTES POR CIUDAD (df_clientes['ciudad'].value_counts().head(5)):
ciudad
Mar del Plata    63
Rosario          55
Posadas          52
Resistencia      50
Córdoba          49
Name: count, dtype: int64

[C] RESUMEN DE PRECIOS, CANTIDADES E INGRESOS (df_ventas)


,precio,cantidad,ingreso_bruto
Conteo,"$ 2.998,00",2998.00,"$ 2.998,00"
Media,"$ 75,29",6.50,"$ 489,36"
Desviación Estándar,"$ 28,74",3.46,"$ 334,28"
Mínimo,"$ 26,00",1.00,"$ 26,30"
Cuartil 25%,"$ 50,03",3.00,"$ 220,92"
Mediana (50%),"$ 75,20",7.00,"$ 418,06"
Cuartil 75%,"$ 100,07",9.00,"$ 709,92"
Máximo,"$ 124,97",12.00,"$ 1.488,12"



PRODUCTOS MÁS VENDIDOS (POR CANTIDAD DE TRANSACCIONES):
categoria
Decoración           1000
Electrodomésticos    1000
Electrónica           998
Name: count, dtype: int64

----------------------------------------------------------
Módulo 4 completado: El formato numérico estricto se ha aplicado.


5 - Calidad de Datos: Identificar valores nulos y duplicados en los conjuntos de datos. Documentar el estado inicial de los datos.

Objetivo

Identificar y documentar el estado inicial de la calidad de los datos en los tres DataFrames, detectando y reportando la presencia de valores nulos, ruidosos (variaciones de vacío) y filas duplicadas.

Proceso
1. Estandarización: Dentro de la función, se define una lista de VALORES_MALOS ('', 'N/A', 'na', etc.) y se utiliza .replace() para convertir todos estos ruidos a un único formato de dato faltante: pd.NA (Nulo).
2. Conteo de Defectos: Se utilizan dos métodos de conteo: a) df.isnull().sum().sum() para contar el total de celdas defectuosas. b) df.isnull().any(axis=1).sum() para contar el total de filas (registros) afectadas por al menos un defecto.
3. Conteo de Duplicados: Se utiliza el método df.duplicated().sum() para contar cuántas filas son copias exactas de otras.
4. Reporte: Se emite un informe estructurado que lista las columnas con valores faltantes, reporta el número de celdas y filas afectadas, y notifica el total de filas duplicadas para cada DataFrame (df_marketing, df_clientes, df_ventas).

Resultado

Un informe detallado que documenta el nivel de "suciedad" de los datos, justificando la posterior fase de limpieza exhaustiva.

In [11]:
# =====================================================
# MÓDULO 5: Calidad de Datos (Diagnóstico Inicial)
# =====================================================

print("\n" + "="*70)
print("INICIANDO MÓDULO 5: REVISIÓN DE CALIDAD (Detective de Datos)")
print("Vamos a revisar qué tan 'limpios' están nuestros datos antes de empezar a trabajar.")
print("Este diagnóstico incluye datos faltantes (NaN), vacíos y filas duplicadas.")
print("="*70)

# Definimos los valores que se consideran "malos" o "vacíos" para el diagnóstico.
VALORES_MALOS = ['', ' ', '-', 'N/A', 'N/D', 'na']


def diagnosticar_calidad(df, nombre):
    """
    Función que revisa los defectos (nulos, vacíos y duplicados) en el DataFrame y reporta
    el total de celdas y el total de registros (filas) afectados.
    """

    print(f"\n--- REPORTE DE {nombre} (Total de filas iniciales: {len(df)}) ---")

    # ----------------------------------------------------------------------
    # A) PRE-PROCESAMIENTO: Convertir Ruidos/Vacíos a NaN
    # ----------------------------------------------------------------------
    # Unificamos todos los formatos de 'dato faltante' bajo el estándar 'NaN'.
    df.replace(VALORES_MALOS, pd.NA, inplace=True)

    # ----------------------------------------------------------------------
    # B) CONTEO DE VALORES INCOMPLETOS O RUIDOSOS
    # ----------------------------------------------------------------------
    nulos_por_columna = df.isnull().sum()
    total_celdas_defectuosas = nulos_por_columna.sum() # Total de celdas con problemas

    # ----------------------------------------------------------------------
    # C) CONTEO DE REGISTROS (FILAS) AFECTADOS
    # ----------------------------------------------------------------------
    # .isnull().any(axis=1): Devuelve True para la fila si AL MENOS una celda es nula.
    # .sum(): Cuenta cuántos True (filas afectadas) hay.
    total_registros_afectados = df.isnull().any(axis=1).sum()


    print("1) FILAS INCOMPLETAS (Nulos, Vacíos o Ruidos):")

    if total_celdas_defectuosas > 0:
        # Mostramos los problemas columna por columna
        print("   ¡ALERTA! Estas columnas tienen datos faltantes/sucios:")

        # Filtramos para mostrar SÓLO las columnas con problemas
        problemas = nulos_por_columna[nulos_por_columna > 0]

        for columna, conteo in problemas.items():
            porcentaje = (conteo / len(df)) * 100
            print(f"   -> En '{columna}': Hay {conteo} celdas defectuosas ({porcentaje:.2f}%).")

        # NUEVO REPORTE CLARO: Diferencia entre celdas y filas
        print(f"\n   -> RESUMEN: Tenemos un total de {total_celdas_defectuosas} celdas defectuosas.")
        print(f"   -> IMPACTO REAL: Estos defectos afectan a {total_registros_afectados} registros (filas).")
    else:
        print("   -> ¡Excelente! No encontramos datos faltantes, vacíos o ruidosos.")

    # ----------------------------------------------------------------------
    # D) CONTEO DE FILAS DUPLICADAS
    # ----------------------------------------------------------------------
    total_duplicados = df.duplicated().sum()

    print("\n2) FILAS DUPLICADAS:")
    if total_duplicados > 0:
        print(f"   ¡CUIDADO! Encontramos {total_duplicados} filas que son COPIAS EXACTAS de otra.")
        print("   Serán eliminadas en la limpieza para evitar contar la misma información dos veces.")
    else:
        print("   -> ¡Bien! No hay filas duplicadas.")


# Ejecutar el diagnóstico para cada una de las copias ('_diag') del Módulo 1.3:
diagnosticar_calidad(df_marketing_diag, "df_marketing")
diagnosticar_calidad(df_clientes_diag, "df_clientes")
diagnosticar_calidad(df_ventas_diag, "df_ventas")

print("\n" + "="*70)
print("MÓDULO 5 COMPLETADO: El estado inicial de los datos ha sido reportado.")


INICIANDO MÓDULO 5: REVISIÓN DE CALIDAD (Detective de Datos)
Vamos a revisar qué tan 'limpios' están nuestros datos antes de empezar a trabajar.
Este diagnóstico incluye datos faltantes (NaN), vacíos y filas duplicadas.

--- REPORTE DE df_marketing (Total de filas iniciales: 90) ---
1) FILAS INCOMPLETAS (Nulos, Vacíos o Ruidos):
   -> ¡Excelente! No encontramos datos faltantes, vacíos o ruidosos.

2) FILAS DUPLICADAS:
   -> ¡Bien! No hay filas duplicadas.

--- REPORTE DE df_clientes (Total de filas iniciales: 567) ---
1) FILAS INCOMPLETAS (Nulos, Vacíos o Ruidos):
   -> ¡Excelente! No encontramos datos faltantes, vacíos o ruidosos.

2) FILAS DUPLICADAS:
   -> ¡Bien! No hay filas duplicadas.

--- REPORTE DE df_ventas (Total de filas iniciales: 3035) ---
1) FILAS INCOMPLETAS (Nulos, Vacíos o Ruidos):
   ¡ALERTA! Estas columnas tienen datos faltantes/sucios:
   -> En 'precio': Hay 2 celdas defectuosas (0.07%).
   -> En 'cantidad': Hay 2 celdas defectuosas (0.07%).

   -> RESUMEN: Tenemos

###Etapa 2: Preprocesamiento y Limpieza de Datos (Clases 5 a 8)

1 - Limpieza de Datos: Limpiar el conjunto de datos eliminando duplicados y caracteres no deseados. Documentar el proceso y los resultados.

Objetivo

Asegurar la consistencia de los datos textuales críticos para evitar errores en las agrupaciones (groupby) y uniones (merge) posteriores.

Proceso
1. Documentación: Se presenta un reporte del estado final de los DataFrames (df_marketing, df_clientes, df_ventas) después de la limpieza inicial (verificando 0 nulos y 0 duplicados).
2. Normalización: Se aplica el método .str.strip() a columnas de texto clave (producto, canal, nombre, ciudad, categoria) para eliminar cualquier espacio en blanco residual al inicio o al final de las cadenas.

Resultado

DataFrames listos para agrupaciones precisas, asegurando que categorías como "TV" no se interpreten como "TV " o " TV".

In [12]:
# =====================================================
# MÓDULO 2.1: Limpieza de Datos (Normalización y Documentación)
# =====================================================

print("Iniciando Módulo 2.1: Documentación y Normalización de la Limpieza.")

# ----------------------------------------------------------------------
# 1. ESTADO DE NULOS/DUPLICADOS (Post-Limpieza de Etapa 1)
# ----------------------------------------------------------------------
# Documentamos el estado actual, limpio por el módulo de limpieza anterior.
def verificar_estado(df, nombre):
    nulos = df.isnull().sum().sum()
    duplicados = df.duplicated().sum()
    estado = "OK" if nulos == 0 and duplicados == 0 else "ALERTA"
    return f"   - {nombre}: Filas: {len(df)}, Nulos: {nulos}, Duplicados: {duplicados} ({estado})"

print("-> El estado de duplicados y nulos se considera limpio después de la Etapa 1.")
print("Dimensiones y Estado Actual:")
print(verificar_estado(df_marketing, "df_marketing"))
print(verificar_estado(df_clientes, "df_clientes"))
print(verificar_estado(df_ventas, "df_ventas"))

# ----------------------------------------------------------------------
# 2. LIMPIEZA ADICIONAL: Normalización de Caracteres (Eliminar espacios)
# ----------------------------------------------------------------------
# Se eliminan espacios en blanco al inicio o final de las cadenas de texto.
# Esto asegura que 'TV' no se agrupe como 'TV ' en el análisis.

print("\n-> Normalizando campos de texto (eliminando espacios extra):")

# Columnas de df_marketing (usando 'producto' y 'canal')
df_marketing['producto'] = df_marketing['producto'].str.strip()
df_marketing['canal'] = df_marketing['canal'].str.strip()

# Columnas de df_clientes (usando 'nombre' y 'ciudad')
df_clientes['nombre'] = df_clientes['nombre'].str.strip()
df_clientes['ciudad'] = df_clientes['ciudad'].str.strip()

# Columnas de df_ventas (usando 'producto' y 'categoria')
df_ventas['producto'] = df_ventas['producto'].str.strip()
df_ventas['categoria'] = df_ventas['categoria'].str.strip()

print("-> Normalización de texto completada en todos los DataFrames.")

print("\nMódulo 2.1 completado: Limpieza documentada y normalización de texto finalizada.")

Iniciando Módulo 2.1: Documentación y Normalización de la Limpieza.
-> El estado de duplicados y nulos se considera limpio después de la Etapa 1.
Dimensiones y Estado Actual:
   - df_marketing: Filas: 90, Nulos: 0, Duplicados: 0 (OK)
   - df_clientes: Filas: 567, Nulos: 0, Duplicados: 0 (OK)
   - df_ventas: Filas: 2998, Nulos: 0, Duplicados: 0 (OK)

-> Normalizando campos de texto (eliminando espacios extra):
-> Normalización de texto completada en todos los DataFrames.

Módulo 2.1 completado: Limpieza documentada y normalización de texto finalizada.


2 - Transformación de Datos: Aplicar filtros y transformaciones para crear una tabla de ventas que muestre solo los productos con alto rendimiento.

Objetivo

Filtrar y analizar el segmento de ventas más valioso para identificar los productos que impulsan la mayor parte del ingreso bruto.

Proceso
1. Definición del Umbral: Se calcula la mediana del ingreso_bruto de todas las transacciones de df_ventas.
2. Filtrado: Se crea un nuevo DataFrame, df_ventas_alto_rendimiento, que incluye solo las filas cuyo ingreso_bruto es mayor o igual al umbral (la mediana), aislando así el 50% superior de las transacciones.
3. Agregación y Ranking: Se agrupa el nuevo segmento de alto rendimiento por producto y se calcula el Ingreso Total, el Total de Unidades y el Total de Transacciones por artículo.
4. Ordenamiento: El resultado se ordena de forma descendente (ingreso_total) para presentar el Ranking Top 5 de productos más rentables dentro de este segmento élite.

Resultado

Un ranking claro de los productos que generan el mayor ingreso en las transacciones más grandes.

In [16]:
# =====================================================
# MÓDULO 2.2: Transformación y Ordenamiento (Alto Rendimiento)
# =====================================================

print("\nIniciando Módulo 2.2: Creación y Ordenamiento del DataFrame de Alto Rendimiento. ⚙️")

# ----------------------------------------------------------------------
# 1. DEFINICIÓN DEL UMBRAL
# ----------------------------------------------------------------------
# Usamos el percentil 80 del 'ingreso_bruto' por transacción como umbral.
# Esto aísla el 20% superior (las transacciones más valiosas) de los datos.

umbral_ingreso = df_ventas['ingreso_bruto'].quantile(0.80)
print(f"-> Umbral de Ingreso Bruto (Percentil 80): {formato_moneda(umbral_ingreso)}")

# ----------------------------------------------------------------------
# 2. APLICACIÓN DEL FILTRO Y CREACIÓN DEL NUEVO DATAFRAME
# ----------------------------------------------------------------------
# Filtramos el DataFrame original para incluir solo las filas donde el ingreso
# es mayor o igual al umbral. Usamos .copy() para crear un DataFrame independiente.

df_ventas_alto_rendimiento = df_ventas[df_ventas['ingreso_bruto'] >= umbral_ingreso].copy()

# ----------------------------------------------------------------------
# 3. AGREGACIÓN Y ORDENAMIENTO POR PRODUCTO
# ----------------------------------------------------------------------
# Agrupamos el nuevo DataFrame de Alto Rendimiento por 'producto' para consolidar
# el rendimiento total por artículo. Luego, calculamos las métricas clave (ingreso
# total, unidades y transacciones) y ordenamos el resultado de mayor a menor
# 'ingreso_total' para establecer un ranking de rentabilidad en este segmento.

df_productos_top = df_ventas_alto_rendimiento.groupby('producto').agg(
    ingreso_total=('ingreso_bruto', 'sum'),
    unidades_vendidas=('cantidad', 'sum'),
    transacciones=('id_venta', 'count')
).reset_index()

# Ordenar los productos de alto rendimiento de mayor a menor ingreso total
df_productos_top.sort_values(by='ingreso_total', ascending=False, inplace=True)

# ----------------------------------------------------------------------
# 4. VISUALIZACIÓN DEL TOP 5
# ----------------------------------------------------------------------
# Esta sección se encarga de presentar los resultados del ranking de alto rendimiento.
# Se calcula la cantidad de productos únicos incluidos en el segmento filtrado.
# Posteriormente, se aplica el formato de moneda a la columna 'ingreso_total'
# y el formato de miles a las columnas 'unidades_vendidas' y 'transacciones'
# para facilitar la lectura del reporte, mostrando únicamente los 5 productos líderes.

filas_filtradas = len(df_productos_top)

print(f"-> El segmento de Alto Rendimiento incluye {filas_filtradas} productos únicos.")

print("\n--- Ranking TOP 5 de Productos de Alto Rendimiento (Por Ingreso Total) ---")

# Aplicamos el estilo de visualización (formato_moneda)
df_productos_top_estilo = df_productos_top.head(5).style.format({
    'ingreso_total': formato_moneda,
    'unidades_vendidas': '{:,.0f}'.format,
    'transacciones': '{:,.0f}'.format
})
display(df_productos_top_estilo)

# Muestra el producto líder
producto_lider = df_productos_top.iloc[0]['producto']
ingreso_lider = df_productos_top.iloc[0]['ingreso_total']
print(f"\nEl producto líder en el segmento de Alto Rendimiento es: {producto_lider}, con un Ingreso Total de {formato_moneda(ingreso_lider)}.")

print("\nMódulo 2.2 completado: Productos de Alto Rendimiento ordenados y analizados. ✅")


Iniciando Módulo 2.2: Creación y Ordenamiento del DataFrame de Alto Rendimiento. ⚙️
-> Umbral de Ingreso Bruto (Percentil 80): $ 787,34
-> El segmento de Alto Rendimiento incluye 30 productos únicos.

--- Ranking TOP 5 de Productos de Alto Rendimiento (Por Ingreso Total) ---


,producto,ingreso_total,unidades_vendidas,transacciones
19,Lámpara de mesa,"$ 37.595,79",393,38
20,Microondas,"$ 35.709,18",362,34
3,Auriculares,"$ 33.930,44",330,33
5,Cafetera,"$ 28.035,10",275,26
9,Cuadro decorativo,"$ 27.385,88",282,28



El producto líder en el segmento de Alto Rendimiento es: Lámpara de mesa, con un Ingreso Total de $ 37.595,79.

Módulo 2.2 completado: Productos de Alto Rendimiento ordenados y analizados. ✅


3 - Agregación: Resumir las ventas por categoría de producto y analizar los ingresos generados.



Objetivo

Resumir el rendimiento total de ventas a nivel macro (categoría) para entender la distribución de ingresos y priorizar inventario/marketing.

Proceso
1. Agregación: Se utiliza la función groupby() en df_ventas sobre la columna categoria.
2. Cálculo de Métricas: Se calculan tres métricas clave para cada categoría: Ingreso Total (sum de ingreso_bruto), Unidades Vendidas (sum de cantidad) y Total de Transacciones (count del número de registros de ventas, típicamente usando una columna como id_venta o producto).
3. Ranking: El DataFrame resultante se ordena de forma descendente por Ingreso Total.

Resultado

Una tabla que muestra qué categorías de productos son las más rentables en términos monetarios, unidades y volumen de ventas, identificando claramente la categoría líder.

In [14]:
# =====================================================
# MÓDULO 2.3: Agregación (Resumen por Categoría)
# =====================================================

print("\nIniciando Módulo 2.3: Agregación y Resumen por Categoría. 📊")

# ----------------------------------------------------------------------
# 1. AGRUPACIÓN Y CÁLCULO DE MÉTRICAS CLAVE
# ----------------------------------------------------------------------
# Agrupamos el DataFrame de ventas por la columna 'categoria'.
# Utilizamos .agg() para aplicar múltiples funciones: 'sum' para valores monetarios/cantidad
# y 'count' para obtener el número de transacciones (filas) por categoría.
df_resumen_categoria = df_ventas.groupby('categoria').agg(
    ingreso_total=('ingreso_bruto', 'sum'),
    unidades_vendidas=('cantidad', 'sum'),
    transacciones=('producto', 'count') # Contamos cuántas filas (transacciones) hay por categoría.
).reset_index() # Mueve 'categoria' del índice a una columna regular.

# ----------------------------------------------------------------------
# 2. ORDENAMIENTO Y ANÁLISIS
# ----------------------------------------------------------------------
# Ordenamos el DataFrame de forma descendente por 'ingreso_total' para obtener el ranking de rentabilidad.
df_resumen_categoria.sort_values(by='ingreso_total', ascending=False, inplace=True)

# ----------------------------------------------------------------------
# 3. APLICACIÓN DE FORMATO Y VISUALIZACIÓN
# ----------------------------------------------------------------------
print("\n--- Ranking de Ingresos Totales por Categoría de Producto (Top 5) ---")

# Aplicamos el formato de moneda para el ingreso y el formato de miles para unidades/transacciones.
df_resumen_estilo = df_resumen_categoria.head(5).style.format({
    'ingreso_total': formato_moneda,
    'unidades_vendidas': '{:,.0f}'.format,
    'transacciones': '{:,.0f}'.format
})

display(df_resumen_estilo)

# Análisis clave: Identificación de la categoría líder.
categoria_top = df_resumen_categoria.iloc[0]
print(f"\nANÁLISIS CLAVE:")
print(f"-> La Categoría Líder es: '{categoria_top['categoria']}'.")
print(f"-> Ingreso Total Generado por esta categoría: {formato_moneda(categoria_top['ingreso_total'])}.")

print("\nMódulo 2.3 completado: Resumen de ventas por categoría generado y analizado.")


Iniciando Módulo 2.3: Agregación y Resumen por Categoría. 📊

--- Ranking de Ingresos Totales por Categoría de Producto (Top 5) ---


,categoria,ingreso_total,unidades_vendidas,transacciones
1,Electrodomésticos,"$ 505.299,63","6,592","1,000"
2,Electrónica,"$ 482.577,80","6,413",998
0,Decoración,"$ 479.216,09","6,490","1,000"



ANÁLISIS CLAVE:
-> La Categoría Líder es: 'Electrodomésticos'.
-> Ingreso Total Generado por esta categoría: $ 505.299,63.

Módulo 2.3 completado: Resumen de ventas por categoría generado y analizado.


4 -  Integración de Datos: Combinar los sets de datos de ventas y marketing para obtener una visión más amplia de las tendencias.:

Objetivo

Combinar la información de ingresos (df_ventas) y costos (df_marketing) para obtener una visión holística de la rentabilidad neta de la empresa (Utilidad) a lo largo del tiempo, facilitando el análisis de tendencias y eficiencia del gasto en marketing.

Proceso
1. Agregación Temporal: Se crea la columna 'periodo' (YYYY-MM) en ambos DataFrames. Se agrupa df_ventas por 'periodo' para sumar el Ingreso Mensual. Se agrupa df_marketing por 'periodo' para sumar el Costo Mensual.
2. Combinación (Merge): Se realiza una unión (pd.merge) de tipo Left Join utilizando 'periodo' como llave. Esto asegura que todos los meses con ingresos estén presentes, rellenando con 0 los meses donde no hubo costos de marketing registrados.
3. Cálculo de Utilidad: Se calcula la métrica clave Utilidad Mensual restando el costo_mensual al ingreso_mensual.
4. Visualización: Se muestra el resultado, ordenado por el mes más reciente, con formato de moneda para facilitar la interpretación de las tendencias de rentabilidad.

Resultado

Un DataFrame integrado que muestra la Utilidad Neta Mensual, permitiendo a la gerencia rastrear la rentabilidad en función directa del gasto en marketing.

In [15]:
# =====================================================
# MÓDULO 2.4: Integración de Datos (Ventas vs. Marketing)
# =====================================================

print("\nIniciando Módulo 2.4: Integración de DataFrames (Ventas y Marketing). 🔗")

# ----------------------------------------------------------------------
# 1. PREPARACIÓN DE df_ventas (Agregación por Mes)
# ----------------------------------------------------------------------
# Creamos una columna 'periodo' (YYYY-MM) para servir como llave de unión.
df_ventas['periodo'] = df_ventas['fecha_venta'].dt.to_period('M').astype(str)

# Agrupamos los ingresos de ventas por mes.
df_ventas_mensuales = df_ventas.groupby('periodo').agg(
    ingreso_mensual=('ingreso_bruto', 'sum')
).reset_index()
print("-> Ventas resumidas mensualmente.")

# ----------------------------------------------------------------------
# 2. PREPARACIÓN DE df_marketing (Agregación por Mes)
# ----------------------------------------------------------------------
# Creamos la columna 'periodo' a partir de la fecha de inicio de la campaña.
df_marketing['periodo'] = df_marketing['fecha_inicio'].dt.to_period('M').astype(str)

# Agrupamos los costos de marketing por mes.
df_marketing_mensual = df_marketing.groupby('periodo').agg(
    costo_mensual=('costo', 'sum')
).reset_index()
print("-> Costos de Marketing resumidos mensualmente.")

# ----------------------------------------------------------------------
# 3. COMBINACIÓN DE DATOS (MERGE)
# ----------------------------------------------------------------------
# Realizamos un LEFT JOIN para mantener todos los meses de ventas y añadir
# los costos de marketing coincidentes.
df_integrado = pd.merge(
    df_ventas_mensuales,
    df_marketing_mensual,
    on='periodo',
    how='left' # Mantiene todos los meses de la tabla de ventas (izquierda)
)

# Rellenamos los meses sin costo de marketing (NaN) con 0.
df_integrado['costo_mensual'] = df_integrado['costo_mensual'].fillna(0)

# ----------------------------------------------------------------------
# 4. CÁLCULO DE LA RENTABILIDAD
# ----------------------------------------------------------------------
# Utilidad (Ganancia) = Ingreso - Costo. Esta es la nueva visión integrada.
df_integrado['utilidad_mensual'] = df_integrado['ingreso_mensual'] - df_integrado['costo_mensual']

# ----------------------------------------------------------------------
# 5. VISUALIZACIÓN
# ----------------------------------------------------------------------
print("\n--- Análisis de Rentabilidad Mensual (Ingreso vs Costo) ---")

# Mostramos el resultado ordenado por periodo descendente (mes más reciente primero).
df_integrado_estilo = df_integrado.sort_values(by='periodo', ascending=False).head(10).style.format({
    'ingreso_mensual': formato_moneda,
    'costo_mensual': formato_moneda,
    'utilidad_mensual': formato_moneda
})

display(df_integrado_estilo)

# Reporte clave
total_utilidad = df_integrado['utilidad_mensual'].sum()
print(f"\nANÁLISIS CLAVE:")
print(f"-> La Utilidad Neta Total (Ingreso - Costo) del periodo integrado es: {formato_moneda(total_utilidad)}.")

# Limpieza final: Se eliminan las columnas 'periodo' temporales de los DataFrames originales.
del df_ventas['periodo']
del df_marketing['periodo']

print("\nMódulo 2.4 completado: DataFrames integrados y cálculo de utilidad mensual realizado.")


Iniciando Módulo 2.4: Integración de DataFrames (Ventas y Marketing). 🔗
-> Ventas resumidas mensualmente.
-> Costos de Marketing resumidos mensualmente.

--- Análisis de Rentabilidad Mensual (Ingreso vs Costo) ---


,periodo,ingreso_mensual,costo_mensual,utilidad_mensual
11,2024-12,"$ 117.631,94","$ 22,92","$ 117.609,02"
10,2024-11,"$ 119.951,79","$ 53,63","$ 119.898,16"
9,2024-10,"$ 112.117,13","$ 33,36","$ 112.083,77"
8,2024-09,"$ 115.787,85","$ 45,59","$ 115.742,26"
7,2024-08,"$ 119.680,15","$ 73,00","$ 119.607,15"
6,2024-07,"$ 116.229,97","$ 66,29","$ 116.163,68"
5,2024-06,"$ 108.480,17","$ 35,72","$ 108.444,45"
4,2024-05,"$ 143.727,25","$ 58,97","$ 143.668,28"
3,2024-04,"$ 128.430,69","$ 29,73","$ 128.400,96"
2,2024-03,"$ 136.779,15","$ 24,37","$ 136.754,78"



ANÁLISIS CLAVE:
-> La Utilidad Neta Total (Ingreso - Costo) del periodo integrado es: $ 1.466.649,94.

Módulo 2.4 completado: DataFrames integrados y cálculo de utilidad mensual realizado.
